# Simple Expreiment: predict winner from gold difference at 10 minute

In [155]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score

## Dataset

Source: https://www.kaggle.com/chuckephron/leagueoflegends

In [156]:
df = pd.read_csv('../data/LeagueofLegends.csv',sep=',')

# preview dataset
df.head(2)

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...


## Dataset preprocessing

Add Winner for each match by merging two original columns:

In [157]:
df['winner'] = np.where(df['bResult']==1, 'blue', 'red')
df.drop(['bResult', 'rResult'], axis = 1, inplace = True)

df['winner'].head()

0    blue
1     red
2    blue
3     red
4    blue
Name: winner, dtype: object

Convert and get list-like gold difference:

In [158]:
from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df['golddiff'].head()

0    [0, 0, -14, -65, -268, -431, -488, -789, -494,...
1    [0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...
2    [0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...
3    [0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...
4    [40, 40, 44, -36, 113, 158, -121, -191, 23, 20...
Name: golddiff, dtype: object


For this task, we only reserve golddiff at 10 min:

In [159]:
df['golddiff'] = df['golddiff'].apply(lambda x : x[9])
df['golddiff'].head()

0   -625
1   -242
2    913
3     16
4    205
Name: golddiff, dtype: int64

Finally, we get our x and y data for training:

In [160]:
x = df['golddiff']
y = df['winner']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

## Train



In [161]:
model_LR = LogisticRegression()
model_LR.fit(np.array(x_train).reshape(-1, 1), y_train)
y_pred = model_LR.predict(np.array(x_test).reshape(-1, 1))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        blue       0.71      0.72      0.71       419
         red       0.65      0.63      0.64       343

    accuracy                           0.68       762
   macro avg       0.68      0.68      0.68       762
weighted avg       0.68      0.68      0.68       762

